In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hipp.aerial.aerial_preprocessing import AerialPreprocessing

In [3]:
images_dir = "/home/godinlu/Documents/history-dataset/1978_09_06_aerial/raw_images/"

preproc = AerialPreprocessing(images_dir)

In [4]:
preproc.create_fiducial_template(corner=True, distance_around_fiducial=70)

In [5]:
preproc.create_fiducial_template(midside=True, distance_around_fiducial=70)

In [7]:
results = preproc.detect_fiducials(grid_size=5, max_workers=5)

Fiducials detection: 100%|██████████| 37/37 [00:17<00:00,  2.06it/s]
